In [18]:
import pandas as pd 
import numpy as np
from scipy.optimize import linprog 
import copy

In [19]:
data = pd.read_csv('data.csv', sep=';')

In [152]:
data.head(2)

,Product Name,Production Unit Name,Currency,Production Plan,Planned Sale price,Price - Model Curr(USD),Material Cost,Marginal profit per unit,Total marginal profit,Min production lot restrictions,Contract minimum,Market maximum
0,"Candies ""Hazelnuts"". 0.25 kg",Candies,TRY,8200.0,31.406,2.293,1.335,0.958,7856.980,100,100,15000
1,"Candies ""Hazelnuts"". 0.25 kg",Candies,EUR,100.0,2.021,2.284,1.335,0.950,94.959,100,0,10000


In [11]:
sum_total_marginal_profit = sum(data['Total marginal profit'])
print(sum_total_marginal_profit)

38158.94800000001


In [12]:
sum_candies = sum(data[data['Production Unit Name'] == 'Candies']['Production Plan'])
sum_lolipops = sum(data[data['Production Unit Name'] == 'Lollipops']['Production Plan'])
sum_dragee = sum(data[data['Production Unit Name'] == 'Dragee']['Production Plan'])

sum_candies, sum_lolipops, sum_dragee

(12000.0, 5000.0, 9999.996000000001)

## Candies

In [21]:
def optimise_candies(method):
    data_1 = data[data['Production Unit Name'] == 'Candies']
    MarginPPU = data_1['Marginal profit per unit']
    c = -data_1['Production Plan']
    c = pd.to_numeric(c)
    B = []
    for b in range(len(data_1)):
        B.append(max(np.array(data_1['Min production lot restrictions'])[b], np.array(data_1['Contract minimum'])[b]))
    B = pd.Series(B)
    bounds = [(b, None) for b in B]
    A_ub = np.diag(MarginPPU)
    b_ub = np.array(data_1['Market maximum'])
    A_eq = np.ones(len(data_1)).reshape(1, -1)
    b_eq = 12000
    res_1 = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method=method)
    return res_1

## Lollipops


In [22]:
def optimise_lollipops(method):
    data_2 = data[data['Production Unit Name'] == 'Lollipops']
    MarginPPU_2 = data_2['Marginal profit per unit']
    c_2= -data_2['Production Plan']
    c_2 = pd.to_numeric(c_2)
    B_2 = []
    for b in range(len(data_2)):
        B_2.append(max(np.array(data_2['Min production lot restrictions'])[b], np.array(data_2['Contract minimum'])[b]))
    B_2 = pd.Series(B_2)

    bounds_2 = [(b, None) for b in B_2]

    A_ub_2 = np.diag(MarginPPU_2)
    b_ub_2 = np.array(data_2['Market maximum'])
    A_eq_2 = np.ones(len(data_2)).reshape(1, -1)
    b_eq_2 = 5000
    res_2 = linprog(c_2, A_ub=A_ub_2, b_ub=b_ub_2, A_eq=A_eq_2, b_eq=b_eq_2, bounds=bounds_2, method=method)
    return res_2

## Dragee 

In [23]:
def optimise_dragee(method):
    data_3 = data[data['Production Unit Name'] == 'Dragee']
    MarginPPU_3 = data_3['Marginal profit per unit']
    c_3= -data_3['Production Plan']
    c_3 = pd.to_numeric(c_3)
    B_3 = []
    for b in range(len(data_3)):
        B_3.append(max(np.array(data_3['Min production lot restrictions'])[b], np.array(data_3['Contract minimum'])[b]))
    B_3 = pd.Series(B_3)

    bounds_3 = [(b, None) for b in B_3]

    A_ub_3 = np.diag(MarginPPU_3)
    b_ub_3 = np.array(data_3['Market maximum'])
    A_eq_3 = np.ones(len(data_3)).reshape(1, -1)
    b_eq_3 = 10000
    res_3 = linprog(c_3, A_ub=A_ub_3, b_ub=b_ub_3, A_eq=A_eq_3, b_eq=b_eq_3, bounds=bounds_3, method=method)
    return res_3

## Result

In [38]:
def optimise(method='interior-point'):
    res_1 = optimise_candies(method)
    res_2 = optimise_lollipops(method)
    res_3 = optimise_dragee(method)
    X = np.concatenate((res_1.x, res_2.x,res_3.x), axis=0)
    copy_of_data = copy.deepcopy(data)
    copy_of_data['Production Plan'] = X
    copy_of_data['Production Plan'] = copy_of_data['Production Plan'].astype(int)

    copy_of_data['Total marginal profit'] = copy_of_data['Production Plan'] * copy_of_data['Marginal profit per unit']
    print('Sum Total marginal profit : ',sum(copy_of_data['Total marginal profit']))

    sum_candies = sum(copy_of_data[copy_of_data['Production Unit Name'] == 'Candies']['Production Plan'])
    sum_lolipops = sum(copy_of_data[copy_of_data['Production Unit Name'] == 'Lollipops']['Production Plan'])
    sum_dragee = sum(copy_of_data[copy_of_data['Production Unit Name'] == 'Dragee']['Production Plan'])

    print('Check out count of elements {} : Candies, {} : Lollipops, {} : Dragee'.format(sum_candies, sum_lolipops, sum_dragee))
    print('Optimisation method : {}'.format(method))
    return copy_of_data

In [39]:
data_4 = optimise()

Sum Total marginal profit :  29478.541
Check out count of elements 11999 : Candies, 4998 : Lollipops, 9999 : Dragee
Optimisation method : interior-point


In [41]:
data_4

,Product Name,Production Unit Name,Currency,Production Plan,Planned Sale price,Price - Model Curr(USD),Material Cost,Marginal profit per unit,Total marginal profit,Min production lot restrictions,Contract minimum,Market maximum
0,"Candies ""Hazelnuts"". 0.25 kg",Candies,TRY,11049,31.406,2.293,1.335,0.958,10584.942,100,100,15000
1,"Candies ""Hazelnuts"". 0.25 kg",Candies,EUR,100,2.021,2.284,1.335,0.950,95.000,100,0,10000
2,"Candies ""Hazelnuts"". 0.25 kg",Candies,USD,100,2.283,2.283,1.335,0.948,94.800,100,100,5000
3,"Candies ""Cashews"". 0.25 kg",Candies,TRY,150,45.777,3.342,1.925,1.417,212.550,100,150,1500
4,"Candies ""Cashews"". 0.25 kg",Candies,EUR,100,2.946,3.329,1.925,1.404,140.400,100,0,800
5,"Candies ""Cashews"". 0.25 kg",Candies,USD,200,3.327,3.327,1.925,1.403,280.600,100,200,1000
6,"Candies ""Pistachios"". 0.25 kg",Candies,TRY,100,48.719,3.557,3.307,0.249,24.900,100,50,1500
7,"Candies ""Pistachios"". 0.25 kg",Candies,EUR,100,3.136,3.543,3.307,0.236,23.600,100,0,900
8,"Candies ""Pistachios"". 0.25 kg",Candies,USD,100,3.541,3.541,3.307,0.234,23.400,100,50,2000
9,"Lollipops ""Chocolate"". 1 kg",Lollipops,TRY,2427,49.584,3.620,2.384,1.236,2999.772,50,200,3000


In [15]:
optimise(method='revised simplex')

Sum Total marginal profit :  29479.499
Check out count of elements 12000 : Candies, 4998 : Lollipops, 9999 : Dragee
Optimisation method : revised simplex


In [16]:
optimise(method='highs')

Sum Total marginal profit :  29479.499
Check out count of elements 12000 : Candies, 4998 : Lollipops, 9999 : Dragee
Optimisation method : highs


In [17]:
optimise(method='simplex')

Sum Total marginal profit :  29479.499
Check out count of elements 12000 : Candies, 4998 : Lollipops, 9999 : Dragee
Optimisation method : simplex
